In [ ]:
from ray.tune.schedulers import PopulationBasedTraining
from ray import tune, air
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.algorithms.ppo import PPO, PPOConfig

from bmstestbedc2f2.envs import MultiAgentBuildingEnv
from bmstestbedc2f2.utils import resolve_path


pbt = PopulationBasedTraining(
    time_attr="training_iteration",
    perturbation_interval=4,
    resample_probability=0.25,
    hyperparam_mutations={
        "lr": tune.uniform(1e-5, 0.1),
        "batch_mode": ["complete_episodes"],
        "train_batch_size": [4000],
        "sgd_minibatch_size": [32, 64, 128, 256, 512],
        "num_sgd_iter": [10, 20, 30],
        "clip_param": tune.uniform(0.1, 0.3),
    },
)

tuner = tune.Tuner(
    PPO,
    param_space=(
        MultiAgentBuildingEnv.get_algo_config(
            PPOConfig()
            .api_stack(
                enable_rl_module_and_learner=False,
                enable_env_runner_and_connector_v2=False,
            )
            .rollouts(
                sample_timeout_s=60,
                num_env_runners=4, 
                rollout_fragment_length='auto',
                # rollout_fragment_length=200,
            )
            .resources(num_gpus=1.),
            env_config=dict(
                bms_system='energyplus',
            ),
        )
    ),
    tune_config=tune.TuneConfig(
        #reuse_actors=True,
        scheduler=pbt,
        num_samples=1,
        metric="env_runners/episode_reward_mean",
        mode="max",
    ),
    run_config=air.RunConfig(
        stop={"training_iteration": 200},
        checkpoint_config=air.CheckpointConfig(
            checkpoint_at_end=True
        ),
        verbose=2,
    )
)
results = tuner.fit()

/home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-22 16:38:16,573	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-01-22 16:38:16,900	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-01-22 16:38:22,328	WARNING deprecation.py:50 -- DeprecationWarning: `rollouts` has been deprecated. Use `AlgorithmConfig.env_runners(..)` instead. This will raise an error in the future!
/home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packages/ray/_private/node.py:1362: ResourceWarning: unclosed file <_io.TextIOWrapper name='/tmp/ray/session_2025-01-22_16-38-22_357299_

(RolloutWorker pid=60293) /usr/lib/python3.11/tempfile.py:1043: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/.energyplus_output_x3vg8f58'>
(RolloutWorker pid=60293)   _warnings.warn(warn_message, ResourceWarning)
(RolloutWorker pid=60292) /usr/lib/python3.11/tempfile.py:1043: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/.energyplus_output_hkhnlo_o'>
(RolloutWorker pid=60293) /usr/lib/python3.11/tempfile.py:1043: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/.energyplus_output_rijx2p0y'>
(RolloutWorker pid=60293) /home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packages/controllables/energyplus/variables.py:396: RuntimeWarning: OutputVariable(OutputVariable.Ref(type='Schedule Value', key='Office_OpenOff_Occ')) requested while Kernel() is running; It may not be available until the next run. More info: https://energyplus.readthedocs.io/en/latest/datatransfer.html#datatransfer.DataExchange.request_variable
(RolloutWork

Trial name,agent_timesteps_total,counters,custom_metrics,env_runners,episode_media,num_agent_steps_sampled,num_agent_steps_sampled_lifetime,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_lifetime,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_healthy_workers,num_in_flight_async_sample_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,timers
PPO_MultiAgentBuildingEnv_4bfe2_00000,160000,"{'num_env_steps_sampled': 20000, 'num_env_steps_trained': 20000, 'num_agent_steps_sampled': 160000, 'num_agent_steps_trained': 160000}",{},"{'episode_reward_max': np.float64(4.558956759103771), 'episode_reward_min': np.float64(3.5911330384479125), 'episode_reward_mean': np.float64(4.00295894558848), 'episode_len_mean': np.float64(4608.0), 'episode_media': {}, 'episodes_timesteps_total': 18432, 'policy_reward_min': {'ART-01-12': np.float64(0.37167016719436274), 'ART-01-07': np.float64(0.3575280315706255), 'ART-01-11a': np.float64(0.3858123028180921), 'ART-01-10': np.float64(0.4648015323826841), 'ART-01-13': np.float64(0.3375223117691081), 'ART-01-09': np.float64(0.4648015323826889), 'ART-01-08': np.float64(0.4989493878079416), 'ART-01-14': np.float64(0.3246698345253709)}, 'policy_reward_max': {'ART-01-12': np.float64(0.49308580363014587), 'ART-01-07': np.float64(0.6640810894949116), 'ART-01-11a': np.float64(0.6216546826237189), 'ART-01-10': np.float64(0.6388125060536645), 'ART-01-13': np.float64(0.596386099182461), 'ART-01-09': np.float64(0.6529546416773826), 'ART-01-08': np.float64(0.5792282757525274), 'ART-01-14': np.float64(0.6105282348062041)}, 'policy_reward_mean': {'ART-01-12': np.float64(0.4499885100390165), 'ART-01-07': np.float64(0.4641306456627473), 'ART-01-11a': np.float64(0.4994859847220785), 'ART-01-10': np.float64(0.5454479254992023), 'ART-01-13': np.float64(0.4641306456627471), 'ART-01-09': np.float64(0.538376857687336), 'ART-01-08': np.float64(0.5348413237814075), 'ART-01-14': np.float64(0.5065570525339449)}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [np.float64(4.558956759103771), np.float64(3.722894525612614), np.float64(4.138851459189625), np.float64(3.5911330384479125)], 'episode_lengths': [4608, 4608, 4608, 4608], 'policy_ART-01-12_reward': [np.float64(0.4691068785688819), np.float64(0.37167016719436274), np.float64(0.4660911907626757), np.float64(0.49308580363014587)], 'policy_ART-01-07_reward': [np.float64(0.4408226073214212), np.float64(0.3575280315706255), np.float64(0.6640810894949116), np.float64(0.3940908542640308)], 'policy_ART-01-11a_reward': [np.float64(0.5963860991824699), np.float64(0.3858123028180921), np.float64(0.6216546826237189), np.float64(0.3940908542640331)], 'policy_ART-01-10_reward': [np.float64(0.6388125060536645), np.float64(0.4989493878079373), np.float64(0.5792282757525233), np.float64(0.4648015323826841)], 'policy_ART-01-13_reward': [np.float64(0.596386099182461), np.float64(0.5838022015503221), np.float64(0.3388119701490969), np.float64(0.3375223117691081)], 'policy_ART-01-09_reward': [np.float64(0.6529546416773826), np.float64(0.4706651165604797), np.float64(0.5650861401287924), np.float64(0.4648015323826889)], 'policy_ART-01-08_reward': [np.float64(0.5539596923112767), np.float64(0.4989493878079416), np.float64(0.5792282757525274), np.float64(0.5072279392538839)], 'policy_ART-01-14_reward': [np.float64(0.6105282348062041), np.float64(0.5555179303028613), np.float64(0.3246698345253709), np.float64(0.5355122105013433)]}, 'sampler_perf': {'mean_raw_obs_processing_ms': np.float64(1.0305325396773055), 'mean_inference_ms': np.float64(6.540494947517361), 'mean_action_processing_ms': np.float64(0.6689045701871533), 'mean_env_wait_ms': np.float64(15.430571555328674), 'mean_env_render_ms': np.float64(0.0)}, 'num_faulty_episodes': 0, 'connector_metrics': {}, 'num_episodes': 4, 'episode_return_max': np.float64(4.558956759103771), 'episod

In [ ]:
best_result = results.get_best_result()
best_result.checkpoint.to_directory(resolve_path('algo_checkpoint'))
best_result

(PPO pid=57755) 2025-01-22 16:35:37,921	ERROR actor_manager.py:804 -- Ray error (ray::RolloutWorker.apply() (pid=57874, ip=192.168.200.249, actor_id=dedd1d90b621e1f07dd735e701000000, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f241765e7d0>)
(PPO pid=57755)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(PPO pid=57755)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(PPO pid=57755)   File "/home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packages/ray/rllib/utils/actor_manager.py", line 194, in apply
(PPO pid=57755)     raise e
(PPO pid=57755)   File "/home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packages/ray/rllib/utils/actor_manager.py", line 183, in apply
(PPO pid=57755)     return func(self, *args, **kwargs)
(PPO pid=57755)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
(PPO pid=57755)   File "/home/AD/user/lab/bmstestbedc2f2/.venv/lib/python3.11/site-packages/ray/rllib/execution/rollout_ops.py", line 102, in <lambda>
(PPO pid=57755)     (lambda w:

AttributeError: 'NoneType' object has no attribute 'to_directory'